# Class level relationships

## Generalization

We need a target project rather than a single file.

In [ ]:
# Clone a target project. This time, we try Mojang/brigadier, a MineCraft program.
!git clone git@github.com:Mojang/brigadier.git

In [ ]:
# First, we need to read many java files.
# from os.path import isfile, join
import os

path = "./brigadier/src/main/java/**/*.java"

from glob import glob


for filename in glob(path, recursive=True):
#     print(filename)
    pass

In [ ]:
javafiles = list(glob(path, recursive=True))
len(javafiles)

In [ ]:
## Parser function
import javalang

def getAST(path: str):
    with open(path, "r") as f:
        code = f.read()
    return javalang.parse.parse(code)

## Get AST one-by-one

In [ ]:
# print(javafiles[0])
myast = getAST(javafiles[14])
# myast

## Detect `extends` and `implements`

In [ ]:
gen_real_dict = {}

for javafile in javafiles:
    myast = getAST(javafile)
    for path, node in javalang.ast.walk_tree(myast):
        if isinstance(node, javalang.tree.ClassDeclaration):
            gen_real_dict[node.name] = { "extends": node.extends.name if node.extends is not None else None,
                                         "implements": list([atype.name for atype in node.implements]) if node.implements is not None else None,
                                         "type": "class"}
#             print("=====")
#             print("Class:", node.name)
#             print("Extends:", node.extends)
#             print("Implements:", node.implements)
        elif isinstance(node, javalang.tree.InterfaceDeclaration):
            gen_real_dict[node.name] = { "extends": node.extends.name if node.extends is not None else None,
                                         "type": "interface"}
#             print("=====")
#             print("Interface:", node.name)
#             print("Extends:", node.extends)
        elif isinstance(node, javalang.tree.AnnotationDeclaration):
            gen_real_dict[node.name] = {}
#             print("=====")
#             print("Annotation:", node.name)
        elif isinstance(node, javalang.tree.EnumDeclaration):
            gen_real_dict[node.name] = { "implements": list([atype.name for atype in node.implements]) if node.implements is not None else None,
                                         "type": "enum"}
#             print("=====")
#             print("Enum:", node.name)
#             print("Implements:", node.implements)    

In [ ]:
# gen_real_dict
for key, value in gen_real_dict.items():
    if value["type"] == "class":
        if value["extends"] is not None:
            gen_real_dict[key]["dangling"] = False
            if value["extends"] in gen_real_dict:
                gen_real_dict[value["extends"]]["dangling"] = False
        if value["implements"] is not None:
            for aiterface in value["implements"]:
                gen_real_dict[key]["dangling"] = False
                if aiterface in gen_real_dict:
                    gen_real_dict[aiterface]["dangling"] = False
    elif value["type"] == "interface":
        if value["extends"] is not None:
            gen_real_dict[key]["dangling"] = False
            if value["extends"] in gen_real_dict:
                gen_real_dict[value["extends"]]["dangling"] = False
#     if value["implements"] is not None and len(value["implements"]) > 1:
#         print(f"{key}:", value["implements"])

for key, value in gen_real_dict.items():
    if "dangling" not in value:
        print(f"{key} is dangling")

### Excercise: Identify dangling interfaces
It should be useful if we can figure out interfaces, which are not implemented by any other classes or is not a parent interface of anohter interface.

In [ ]:
gen_real_dict

## Store generalization relationships in a data structure

In [ ]:
for key, value in gen_real_dict.items():
    if value["extends"] is not None:
        parent = value["extends"]
        print(f"{key} -> {parent}")

### Excercise: making inverse relationships of generalization
Currently, we have `subclass -> parentclass`. We can computer `parentclass -> subclass`.

In addition, we can create a forest of class hierarchy in a project by using the above information.

# Association

## Composition
It is quite challenging to figure out compositions in Java as it needs dynamic information.

## Aggregation

#### Fields (member variables)

Collect field information of each class.

In [ ]:
classes = {}

for javafile in javafiles:
    myast = getAST(javafile)
    for path, node in javalang.ast.walk_tree(myast):
        if isinstance(node, javalang.tree.ClassDeclaration):
           classes[node.name] = {"ast_node": node}

In [ ]:
for cname, props in classes.items():
    fields = {}
    for field in props["ast_node"].fields:
        type_names = []
        if isinstance(field.type, javalang.tree.BasicType):
            type_names.append(field.type.name)
        elif isinstance(field.type, javalang.tree.ReferenceType):
            type_names.append(field.type.name)
            if field.type.arguments is not None:
                for path, node in javalang.ast.walk_tree(field.type.arguments):
                    if isinstance(node, javalang.tree.ReferenceType):
                        type_names.append(node.name)
        for decl in field.declarators:
            fields[decl.name] = {
                "static": True if "static" in field.modifiers else False,
                "types": type_names
            }
    classes[cname]["fields"] = fields

Check fields for each class.

In [ ]:
for cname, props in classes.items():
    fields = props["fields"]
    print(f"{cname}:")
    for f in fields:
        print(f"\t{f}: {fields[f]}")

#### Find aggregation between classes



In [ ]:
# Iterate member variable information (type information)

## Dependencies

### Methods

Collect type information from method bodies for dependency relationships.

#### Constructors

In [ ]:
for cname, props in classes.items():
#     print(f"{cname}:")
    consts = {}
    for const in props["ast_node"].constructors:
        typeformal = []
        for param in const.parameters:
            typeformal.append(param.type.name)
        ckey = "+".join(typeformal) if len(typeformal) > 0 else "void"
        consts[ckey] = const
#         print("\tconstructor:", ckey)
    classes[cname]["constructors"] = consts



#### Other member methods

In [ ]:
for cname, props in classes.items():
#     print(f"{cname}:")
    methods = {}
    for method in props["ast_node"].methods:
        typeformal = []
        for param in method.parameters:
            typeformal.append(param.type.name)
        typekey = "+".join(typeformal) if len(typeformal) > 0 else "void"
        ckey = f"{method.name}({typekey})"
        methods[ckey] = method
#         print("\tmethod:", ckey)
    classes[cname]["methods"] = methods

#### Return

Collect the type information of the return type of a method.

In [ ]:
def collect_type_strings(node):
    type_strings = []
    for path, node in javalang.ast.walk_tree(node):
        if isinstance(node, javalang.tree.BasicType):
            type_strings.append(node.name)
        if isinstance(node, javalang.tree.ReferenceType):
            type_strings.append(node.name)
    return set(type_strings)

In [ ]:
for cname, props in classes.items():
#     print(f"{cname}:")
    dependent_types = set()
    for mkey, node in props["methods"].items():
        ret_type_set = collect_type_strings(node.return_type) if node.return_type is not None else set()
        dependent_types = dependent_types.union(ret_type_set)
    classes[cname]["dependency"] = dependent_types
#     print(f"\tdependency: {dependent_types}")

#### Formal parameters

In [ ]:
for cname, props in classes.items():
    print(f"{cname}:")
# ["constructors"]
    props["dependency"]
    for mkey, node in props["constructors"].items():
        for param in node.parameters:
            props["dependency"] = props["dependency"].union(collect_type_strings(param))
    for mkey, node in props["methods"].items():
        for param in node.parameters:
            props["dependency"] = props["dependency"].union(collect_type_strings(param))
    d_type = props["dependency"]
    print(f"\tdependency: {d_type}")

#### Statements in method bodies